# Resale Flat Prices

## Summary
- Data source: [Resale flat prices based on registration date from Jan-2017 onwards](https://beta.data.gov.sg/datasets/d_8b84c4ee58e3cfc0ece0d773c8ca6abc/view)
  - Time range: `Jan 2017 - Apr 2024`
  - The approximate floor area includes any recess area purchased, space adding item under HDB’s upgrading programmes, roof terrace, etc.

In [5]:
df = spark.read.csv('ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv', header=True, inferSchema=True)
df.show()

Py4JJavaError: An error occurred while calling o72.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 11) (172.31.0.3 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/home/jovyan/work/sg-resale-flat-prices/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala